In [ ]:
!pip install -q git+https://github.com/ficstamas/data-mining.git

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True)

# 1. Graphs

A **graph** $G = (V, E)$ consists of:
- A set of **vertices** (or **nodes**) $V$.
- A set of **edges** $E \subseteq V \times V$.

## 1.1 Types of graphs

- **Undirected graph**: If $(u, v) \in E$ implies $(v, u) \in E$. Edges have no direction.
- **Directed graph (digraph)**: Edges have orientation. $(u, v)$ and $(v, u)$ are distinct.
- **Weighted graph**: Each edge has an associated weight $w(u, v)$ (cost, capacity, similarity, etc.).
- **Unweighted graph**: All edges are considered to have the same weight (often 1).
- **Simple graph**: No self-loops $(v, v)$ and no multiple edges between the same pair of vertices.
- **Bipartite graph**: Vertices can be split into two sets $V_1, V_2$ such that every edge connects a node in $V_1$ to a node in $V_2$.

## 1.2 Graph representations

Let $n = |V|$ be the number of nodes.

### 1.2.1 Adjacency matrix

An **adjacency matrix** $A$ is an $n \times n$ matrix where
$$ A_{ij} = \begin{cases}
1 & \text{if there is an edge from node } i \text{ to node } j, \\
0 & \text{otherwise.}
\end{cases} $$

For weighted graphs, $A_{ij}$ can store the weight of the edge $(i, j)$ instead of 0/1.

### 1.2.2 Adjacency list

An **adjacency list** keeps, for each node, a list of its neighbors:

- For a directed graph: neighbors it has outgoing edges to.
- For an undirected graph: neighbors it is connected to.

Adjacency lists are usually more memory-efficient for sparse graphs (when there are relatively few edges).

In [ ]:
# Example: create a small directed graph and show different representations

# Nodes: 0, 1, 2, 3
# Edges: 0->1, 0->2, 1->2, 2->0, 2->3, 3->3
edges = [(0, 1), (0, 2), (1, 2), (2, 0), (2, 3), (3, 3)]
n = 4

In [ ]:
# Adjacency matrix representation
# TODO: make an adjacency matrix from the edges defined above
A = None

print("Adjacency matrix A:")
print(A)

In [ ]:
# Adjacency list representation (using a dict of lists)
# TODO: make and adjacency list from the edges defined above
adj_list = None

print("\nAdjacency list:")
for node, neigh in adj_list.items():
    print(f"{node}: {neigh}")

In [ ]:
# Visualization of the graph using networkx
G = nx.DiGraph()
G.add_edges_from(edges)

plt.figure()
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, arrows=True)
plt.title("Example directed graph")
plt.show()

# 2. Markov processes

A **Markov chain** (discrete-time, finite-state) is a sequence of random variables $X_0, X_1, X_2, \dots$ taking values in a finite state space $S = \{1, \dots, n\}$ such that
$$ \mathbb{P}(X_{t+1} = j \mid X_t = i, X_{t-1}, \dots, X_0) = \mathbb{P}(X_{t+1} = j \mid X_t = i). $$
That is, the future depends only on the present state, not on the past history.

The transition probabilities are encoded in an $n \times n$ **transition matrix** $P$ with entries
$$ P_{ij} = \mathbb{P}(X_{t+1} = j \mid X_t = i). $$

## 2.1 Row-stochastic matrices

A matrix $P$ is called **row-stochastic** if:
- All entries are non-negative: $P_{ij} \ge 0$.
- Each row sums to 1: $\sum_j P_{ij} = 1$ for all $i$.

Transition matrices of Markov chains are row-stochastic matrices.

In [ ]:
def is_row_stochastic(P, tol=1e-8):
    P = np.array(P, dtype=float)
    row_sums = P.sum(axis=1)
    return np.allclose(row_sums, np.ones(P.shape[0]), atol=tol)

# Example transition matrix
P_example = np.array([
    [0.1, 0.6, 0.3],
    [0.0, 1.0, 0.0],
    [0.5, 0.0, 0.5]
])

print("P_example:\n", P_example)
print("Is row-stochastic?", is_row_stochastic(P_example))

## 2.2 Stationary distribution

A **stationary distribution** of a Markov chain with transition matrix $P$ is a row vector $\pi$ such that:
$$ \pi \ge 0, \quad \sum_i \pi_i = 1, \quad \text{and} \quad \pi P = \pi. $$

Intuitively, if $X_0$ is distributed according to $\pi$, then $X_t$ will also be distributed according to $\pi$ for all $t$. For many Markov chains, under mild conditions, the distribution of $X_t$ converges to a unique stationary distribution as $t \to \infty$.

We can compute $\pi$ as the **left eigenvector** of $P$ corresponding to eigenvalue 1, or by solving a linear system.

In [ ]:
def stationary_distribution(P):
    """Compute a stationary distribution of a finite Markov chain.
    Returns the normalized left eigenvector of P with eigenvalue 1.
    """
    P = np.array(P, dtype=float)
    # TODO: find the stationary vector using eigen decomposition
    return pi

pi_example = stationary_distribution(P_example)
print("Stationary distribution pi:", pi_example)
print("Check pi * P:", pi_example @ P_example)

# 3. Markov processes and random walks on graphs

A natural way to construct a Markov chain from a graph is via a **random walk**:

- Suppose we have a directed graph with adjacency matrix $A$.
- For each node $i$, let $\text{outdeg}(i)$ be the number of outgoing edges from $i$.
- In a simple random walk, from node $i$ we choose one of its outgoing neighbors uniformly at random.

This defines a transition matrix $P$ by
$$ P_{ij} = \begin{cases}
\frac{1}{\text{outdeg}(i)} & \text{if there is an edge } i \to j, \\
0 & \text{otherwise.}
\end{cases} $$

If some node has no outgoing edges (a **sink**), we need to handle it specially (e.g., by adding self-loops or teleportation).

In [ ]:
def random_walk_transition_matrix(A):
    """Given an adjacency matrix A, construct the random-walk transition matrix P.
    If a row has all zeros (sink node), we keep the row as all zeros here
    """
    A = np.array(A, dtype=float)
    n = A.shape[0]
    P = np.zeros_like(A)
    for i in range(n):
        outdeg = A[i].sum()
        if outdeg > 0:
            P[i] = A[i] / outdeg
        else:
            P[i] = 0  # sink
    return P

P_rw = random_walk_transition_matrix(A)
print("Random walk transition matrix P_rw:\n", P_rw)
print("Is P_rw row-stochastic?", is_row_stochastic(P_rw))

In [ ]:
M = P_rw
# Fix random seed
np.random.seed(42)

# Generate 5 random row vectors from [0,1]
P = np.random.rand(5, 4)

# Normalize rows to become probability distributions
P = P / P.sum(axis=1, keepdims=True)

# Power iteration for 15 steps
for k in range(15):
    P = P @ M

P

In [ ]:
stationary_distribution(M)

# 4. PageRank

PageRank is an algorithm originally used by Google to rank web pages. It can be interpreted as the stationary distribution of a modified random walk on the web graph.

## Base algorithm

Consider a directed graph (the web). A "random surfer":
1. With probability $\alpha$ (the **damping factor**, typically 0.85), follows an outgoing link from the current page uniformly at random.
2. With probability $1 - \alpha$, **teleports** to a random page (chosen according to some teleportation distribution, often uniform).

Let $P$ be the random-walk transition matrix constructed from the graph, and let $v$ be the teleportation distribution (a row vector that sums to 1, if $v$ is non-uniform then we call that a Personalized PageRank). Then the PageRank transition matrix is
$$ G = \alpha P + (1 - \alpha) \mathbf{1} v, $$
where $\mathbf{1}$ is a column vector of all ones. The PageRank vector $\pi$ is the stationary distribution of $G$:
$$ \pi G = \pi. $$

We often compute $\pi$ via **power iteration**:
$$ \pi^{(t+1)} = \pi^{(t)} G, $$
starting from some initial distribution $\pi^{(0)}$ and iterating until convergence.

In [ ]:
def pagerank(A, alpha=0.85, v=None, tol=1e-8, max_iter=100):
    """Compute PageRank vector for a graph with adjacency matrix A.
    Uses the random surfer model with damping factor alpha and uniform teleportation if 'v' is not provided.
    """
    A = np.array(A, dtype=float)
    n = A.shape[0]
    # Random walk matrix (handle sinks by leaving rows as zeros, then teleportation fixes them)
    P = random_walk_transition_matrix(A)

    # Teleportation distribution: uniform over all nodes
    # TODO define 'v'

    # Normalize v to be a probability distribution
    if v.sum() == 0:
        raise ValueError("Personalization vector v must have non-zero sum.")
    v = v / v.sum()

    # Handle sinks: replace zero rows of P with v
    for i in range(n):
        if np.allclose(P[i], 0):
            P[i] = v

    # Power iteration
    pi = np.ones(n) / n  # initial distribution
    for _ in range(max_iter):
        # TODO: define the iteration step
        pi_new = 
        if np.linalg.norm(pi_new - pi, 1) < tol:
            pi = pi_new
            break
        pi = pi_new
    return pi

In [ ]:
pagerank_scores = pagerank(A, alpha=0.85)
print("PageRank scores for nodes 0,1,2,3:")
print(pagerank_scores, "(sum=", pagerank_scores.sum(), ")")

In [ ]:
pagerank_scores = pagerank(A, alpha=0.85, v=np.array([0.1, 0.5, 0.3, 0.1]))
print("Personalized PageRank scores for nodes 0,1,2,3:")
print(pagerank_scores, "(sum=", pagerank_scores.sum(), ")")

In [ ]:
from datamining.plots.pagerank import plot_pagerank_iteration

plot_pagerank_iteration(A, v=np.array([0.1, 0.59, 0.3, 0.01]))

# 5. HITS (Hyperlink-Induced Topic Search)

HITS is another link analysis algorithm. It assigns two scores to each node:
- **Authority score**: how authoritative the node is (many good hubs point to it).
- **Hub score**: how good the node is as a hub (it points to many good authorities).

Given adjacency matrix $A$ (where $A_{ij} = 1$ if there is a link from $i$ to $j$), the HITS algorithm iteratively updates:

$$ a \leftarrow A^T h, \quad h \leftarrow A a, $$

where $a$ is the vector of authority scores and $h$ is the vector of hub scores. After each step, we typically normalize the vectors (e.g., divide by their Euclidean norm) to avoid divergence.

Under certain conditions, this iteration converges to the principal eigenvectors corresponding to authorities and hubs.

In [ ]:
def hits(A, max_iter=100, tol=1e-8):
    """Compute HITS authority and hub scores for adjacency matrix A."""
    A = np.array(A, dtype=float)
    n = A.shape[0]
    # Initialize scores
    h = np.ones(n)
    a = np.ones(n)

    for _ in range(max_iter):
        # TODO: finish the iteration step
        a_new = 
        h_new =

        if np.linalg.norm(a_new - a) + np.linalg.norm(h_new - h) < tol:
            a, h = a_new, h_new
            break

        a, h = a_new, h_new

    return a, h

auth_scores, hub_scores = hits(A)
print("Authority scores:", auth_scores)
print("Hub scores:", hub_scores)

## Visualizing HITS scores

We can visualize authority and hub scores on our example graph. We'll use node color for authorities and node size for hubs.

In [ ]:
plt.figure()
pos = nx.spring_layout(G, seed=42)

# Scale scores for plotting
node_sizes = 2000 * (hub_scores - hub_scores.min() + 0.1)
node_colors = auth_scores

nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap='viridis')
nx.draw_networkx_edges(G, pos, arrows=True)
nx.draw_networkx_labels(G, pos)
plt.colorbar(nodes, label='Authority score')
plt.title("HITS: node size = hub score, color = authority score")
plt.axis('off')
plt.show()